In [32]:
import torch
PATH = "D:\TarunDocs\Git\Facemask_model_trained\Facemask_VGG16.pth"

# Load
model = torch.load(PATH)
model.eval()

RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 2.00 GiB total capacity; 570.08 MiB already allocated; 259.93 MiB free; 868.00 MiB reserved in total by PyTorch)

In [5]:
import os
import numpy as np
import torchvision
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt

## Adding testfolder

In [7]:
Newpath = 'D:/TarunDocs/Kaggle'

In [8]:
transform_val = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [9]:
testpath = os.path.join(Newpath,'test/')

In [10]:
test_data = datasets.ImageFolder(testpath,transform=transform_val)

In [11]:
batch_size = 20
num_workers=0

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)

In [12]:
loaders = {'test': test_loader}

In [13]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    # set the module to evaluation mode
    model.eval()

    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [17]:
import torch.optim as optim
import torch.nn as nn

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
def get_optimizer_transfer(model):
    optimizer = torch.optim.Adam(model.classifier.parameters(), lr = 0.001)
    return optimizer

In [20]:
use_cuda = torch.cuda.is_available()

In [21]:
test(loaders, model, criterion, use_cuda)

D:\Anaconda\envs\DLenv\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
D:\Anaconda\envs\DLenv\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Test Loss: 0.489627


Test Accuracy: 99% (7536/7553)


In [27]:
from PIL import Image
img = Image.open('facemaskimage.jpg')
data = transform_val(img)
dataload = torch.utils.data.DataLoader('facemaskimage.jpg')
data = transform_val(img)

In [30]:
import io
import json
import torch

from torchvision import transforms
from PIL import Image
#from flask import Flask, jsonify, request

In [58]:
Classes = ['with Mask', 'without mask']

In [54]:
model.eval() # Evaluation mode, IMPORTANT

def transform_image(image_bytes): 
    # We will recieve the image as bytes
    transform_image = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
    ])
    image = Image.open(io.BytesIO(image_bytes))
    return transform_image(image).unsqueeze(0)

def get_prediction(image_bytes):
    tensor = transform_image(image_bytes)
    tensor = tensor.cuda()
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = y_hat.item()
    
    return Classes[predicted_idx]

In [60]:
img = Image.open(r"C:\Users\\admin\Pictures\TARUNVIGNESH.jpg")



### Converting img to bytes

In [61]:
with io.BytesIO() as buf:
    img.save(buf, 'jpeg')
    image_bytes = buf.getvalue()

In [62]:
print(get_prediction(image_bytes))

without mask
